In [7]:
import os
import pickle
import sys

sys.path.append(os.path.join(os.getenv("HOME"), "gpuaffman_networks/"))
import general_network_visualization

In [3]:
data_dir = "/home/trevor/boolean_network_data/memory_evolution_results/2023-02-13-17-32-37"
with open(os.path.join(data_dir,"best_populations.pk"), "rb") as f:
    populations = pickle.load(f)


In [4]:
populations = [(x[0][0, ...], x[1][0, ...], x[2][0, ...]) for x in populations]

In [ ]:
def graph_from_ragged_spec(connections, used_connections):
    pass

In [ ]:
graphs = [graph_from_ragged_spec(x[1], x[2]) for x in populations]
